### 1단계: Colab 환경 설정

의존성 충돌을 피하고 재현 가능한 분석 환경을 구축하기 위해 Conda를 사용하여 격리된 가상 환경을 만드는 것이 가장 좋은 방법입니다. `condacolab` 라이브러리를 사용하면 Colab 노트북에서 Conda를 쉽게 사용할 수 있습니다.

먼저 `condacolab`을 설치하고 초기화합니다. **이 셀을 실행하면 런타임이 자동으로 다시 시작됩니다.**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Conda 환경 생성

In [ ]:
%%bash
conda create -n canyaenv python=3.9 -y

### 2단계: CANYA 설치

`canyaenv` 환경에 CANYA를 설치합니다. Lehner 연구소 GitHub 저장소에서 pip로 설치합니다.

**참고:** 각 `%%bash` 셀은 별도의 셸 세션이므로 항상 `conda activate canyaenv`가 필요합니다.

In [ ]:
%%bash
source activate canyaenv
python -m pip install --no-cache-dir https://github.com/lehner-lab/canya/tarball/master

### 4단계: 다양한 옵션으로 CANYA 실행하기

#### 4.1. 기본 실행

In [ ]:
%%bash
source activate canyaenv
canya --input canya_input.txt --output basic_out.txt

#### 4.2. 요약 실행 (`--summarize mean`)

In [ ]:
%%bash
source activate canyaenv
canya --input canya_input.txt --output summarized_out.txt --summarize mean

#### 4.3. 앙상블 모드 실행 (`--mode ensemble`)

In [ ]:
%%bash
source activate canyaenv
canya --input canya_input.txt --output ensemble_out.txt --summarize mean --mode ensemble

### 5단계: 결과 시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    df_results = pd.read_csv('ensemble_out.txt', sep='\t')
    print("CANYA 실행 결과:")
    print(df_results)

    df_sorted = df_results.sort_values('Mean_Score', ascending=False).reset_index(drop=True)
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    sns.barplot(
        x='ID', y='Mean_Score', data=df_sorted,
        ax=ax, palette='viridis', edgecolor='black'
    )

    ax.errorbar(
        x=df_sorted.index,
        y=df_sorted['Mean_Score'],
        yerr=df_sorted['Std_Dev_Score'],
        fmt='none', c='black', capsize=5
    )

    ax.set_title('CANYA Nucleation Score for CDR-H3 Sequences (Ensemble Mode)', fontsize=16)
    ax.set_xlabel('Sequence ID', fontsize=12)
    ax.set_ylabel('Predicted Nucleation Score (Mean ± Std. Dev.)', fontsize=12)
    ax.set_ylim(0, 1)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("출력 파일 'ensemble_out.txt'을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")